In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

from sklearn.model_selection import train_test_split

import re
import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/Colab Notebooks/data/emoji_tweets

/content/drive/MyDrive/Colab Notebooks/data/emoji_tweets


In [ ]:
!pip install sentence-transformers

In [ ]:
# glove_dir = './glove.6B.300d.txt'
# input_dim = 300

# vocab = {}
# with open(glove_dir, encoding='utf8') as f:
#     for line in f:
#         values = line.split()
#         word = values[0]
#         vector = np.asarray(values[1:], "float32")
#         vocab[word] = vector
# f.close()

# print('Found %s word vectors.' %len(vocab))

Found 400000 word vectors.


In [ ]:
# max_len = 50
# zero_padding = [0]*input_dim

# def get_embeddings(text, emb="LSTM"):
#     if emb == "DNN":
#         embedding = [0]*input_dim
#         i = 0
#         for word in text.split(' '):
#             if word in vocab:
#                 i += 1
#                 embedding += vocab[word]

#         if i != 0:
#             embedding /= i
#     elif emb == "LSTM":
#         embedding = []
#         i = 0
#         for word in text.split(' '):
#             if i == max_len:
#                 break
#             if word in vocab:
#                 i += 1
#                 embedding.append(vocab[word])
        
#         while i < max_len:
#             i += 1
#             embedding.append(zero_padding)
            
#     return embedding

In [ ]:
def clean(text):
    """ Function to clean the text """
    text = text.lower()
    text = re.sub(r'^https?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE)
    texter = re.sub(r"<br />", " ", text)
    texter = re.sub(r"&quot;", "\"",texter)
    texter = re.sub('&#39;', "\"", texter)
    texter = re.sub('\n', " ", texter)
    texter = re.sub(' u '," you ", texter)
    texter = re.sub('`',"", texter)
    texter = re.sub(' +', ' ', texter)
    texter = re.sub(r"(!)\1+", r"!", texter)
    texter = re.sub(r"(\?)\1+", r"?", texter)
    texter = re.sub('&amp;', 'and', texter)
    texter = re.sub('\r', ' ',texter)
    
    # Remove numbers from string
    texter = re.sub(pattern=r"[+-]?\d+(?:\.\d+)?", repl="", string=texter, count=0, flags=0)
    texter = texter.replace("  ", " ")
    clean = re.compile('<.*?>')
    texter = texter.encode('ascii', 'ignore').decode('ascii')
    texter = re.sub(clean, '', texter)
    texter = re.sub(r'[^\w\s]', '', texter)
    if texter == "":
        texter = ""
    return texter

In [ ]:
# def transform(X, emb="LSTM"):
#     embeddings = []
#     for item in X:
#         item = clean(item)
#         embedding = get_embeddings(item, emb)
#         embeddings.append(embedding)
    
#     return embeddings

In [ ]:
inputs_file = "./tweets_training.txt"
labels_file = "./tweets_training_labels.txt"

X = []
with open(inputs_file, encoding='utf8') as f:
    for line in f:
        X.append(line)
f.close()
# print(len(X))

y = []
with open(labels_file, encoding='utf8') as f:
    for line in f:
        y.append(int(line))
f.close()



In [ ]:
cleaned_x = []
for line in X:
  cleaned_x.append(clean(line))

In [ ]:
# X = X[:1000]
# y = y[:1000]

In [ ]:
from sentence_transformers import SentenceTransformer
sbert_model = SentenceTransformer('bert-base-nli-mean-tokens', device='cuda')

In [ ]:
sentence_embeddings = sbert_model.encode(cleaned_x, show_progress_bar=True)
X = sentence_embeddings

Batches:   0%|          | 0/11028 [00:00<?, ?it/s]

In [ ]:
# Hyperparameters
num_epochs = 10
batch_size = 64
learning_rate = 0.0001
dropout = 0.25

In [ ]:
# X = transform(X, "DNN")
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)

X_tr = torch.tensor(X_train, dtype=torch.float)
y_tr = torch.tensor(y_train)
train = TensorDataset(X_tr, y_tr)
trainloader = DataLoader(train, batch_size=batch_size)

X_te = torch.tensor(X_test, dtype=torch.float)
y_te = torch.tensor(y_test)
test = TensorDataset(X_te, y_te)
testloader = DataLoader(test)

In [ ]:
class Net(nn.Module):
    
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(768, 500)
        self.hidden1 = nn.Linear(500, 500)
        self.hidden2 = nn.Linear(500, 500)
        self.fc2 = nn.Linear(500, 20)
        self.dropout = nn.Dropout(0.25)
        
        self.batchnorm1 = nn.BatchNorm1d(500)
        self.batchnorm2 = nn.BatchNorm1d(500)
        self.batchnorm3 = nn.BatchNorm1d(500)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.batchnorm1(x)
        x = self.dropout(x)
        x = F.relu(self.hidden1(x))
        x = self.batchnorm2(x)
        x = self.dropout(x)
        x = F.relu(self.hidden2(x))
        x = self.batchnorm3(x)
        x = self.dropout(x)
#         x = self.fc2(x)
        x = F.softmax(self.fc2(x), dim=1)
        return x

net = Net()

In [ ]:
# Loss function
criterion = nn.CrossEntropyLoss()

# create your optimizer
optimizer = optim.Adam(net.parameters(), lr=learning_rate)

In [ ]:
# Training
net.train()
for epoch in range(num_epochs):
    for i, data in enumerate(trainloader):
        inputs, labels = data

        outputs = net(inputs)
        loss = criterion(outputs, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 500 == 0:
            print ('Epoch [%d/%d], Step [%d/%d], Loss: %.4f'
                   %(epoch+1, num_epochs, i+1, len(trainloader), loss.data))

In [ ]:
# Testing
total, correct = 0, 0

net.eval()
outputs = net(X_te)

_, predicted = torch.max(outputs, 1)

total += y_te.size(0)
correct += (predicted == y_te).sum()

print(f'Accuracy of the model is: {100*correct/total:.2f}%')

In [94]:
X.shape

(352867, 768)

In [95]:
X_lstm = X.reshape(X.shape[0], 1, X.shape[1])
X_lstm.shape

(352867, 1, 768)

In [96]:
# Hyperparameters
num_epochs = 3
batch_size = 256
learning_rate = 0.0005
dropout = 0.5

In [101]:
class LSTM(nn.Module): 
    def __init__(self, num_classes, input_size, hidden_size, num_layers, seq_length, dropout):
        super(LSTM, self).__init__()
        self.num_classes = num_classes #number of classes
        self.num_layers = num_layers #number of layers
        self.input_size = input_size #input size
        self.hidden_size = hidden_size #hidden state
        self.seq_length = seq_length #sequence length

        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size,
                          num_layers=num_layers, dropout=dropout, batch_first=True).to('cuda') #lstm
        self.fc1 =  nn.Linear(hidden_size, 128) #fully connected 1
        self.fc2 = nn.Linear(128, num_classes) #fully connected last layer

    def forward(self, x):
        h_0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to('cuda') #hidden state
        c_0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to('cuda') 
        
        # Propagate input through LSTM
        x, _ = self.lstm(x, (h_0, c_0)) #lstm with input, hidden, and internal state
        
        # Flatten lstm output
        x = x[:, -1, :]
        
        x = F.relu(self.fc1(x))
        x = F.softmax(self.fc2(x), dim=1)
        return x

lstm = LSTM(20, 768, 128, 2, 768, dropout).to('cuda')

In [102]:
## LSTM
# X = transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_lstm, y, test_size=0.15, random_state=42)


X_tr = torch.tensor(X_train, dtype=torch.float)
y_tr = torch.tensor(y_train)
train = TensorDataset(X_tr, y_tr)
X_tr = X_tr.to(device='cuda')
y_tr = y_tr.to(device='cuda')
trainloader = DataLoader(train, batch_size=batch_size)

X_te = torch.tensor(X_test, dtype=torch.float)
y_te = torch.tensor(y_test)
X_te = X_te.to(device='cuda')
y_te = y_te.to(device='cuda')
test = TensorDataset(X_te, y_te)
testloader = DataLoader(test)

print(len(y_tr))
print(X_tr.size())

299936
torch.Size([299936, 1, 768])


In [103]:
# Loss function
criterion = nn.CrossEntropyLoss()

# create your optimizer
optimizer = optim.Adam(lstm.parameters(), lr=learning_rate)

In [104]:
# Training
# lstm = lstm.to(device="cuda")
# trainloader.to(device="cuda")
for epoch in range(num_epochs):
    running_loss = 0
    for i, data in enumerate(trainloader):
        inputs, labels = data
        inputs = inputs.to('cuda')
        labels = labels.to('cuda')

        outputs = lstm(inputs)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        optimizer.zero_grad()

        running_loss += loss.item()
        
        if (i+1) % 250 == 0:
            print ('Epoch [%d/%d], Step [%d/%d], Loss: %.4f'
                   %(epoch+1, num_epochs, i+1, len(X_tr)//batch_size, loss.data))

Epoch [1/3], Step [250/1171], Loss: 2.8282
Epoch [1/3], Step [500/1171], Loss: 2.8627
Epoch [1/3], Step [750/1171], Loss: 2.8465
Epoch [1/3], Step [1000/1171], Loss: 2.7889
Epoch [2/3], Step [250/1171], Loss: 2.7766
Epoch [2/3], Step [500/1171], Loss: 2.8294
Epoch [2/3], Step [750/1171], Loss: 2.8388
Epoch [2/3], Step [1000/1171], Loss: 2.7756
Epoch [3/3], Step [250/1171], Loss: 2.7599
Epoch [3/3], Step [500/1171], Loss: 2.8191
Epoch [3/3], Step [750/1171], Loss: 2.8369
Epoch [3/3], Step [1000/1171], Loss: 2.7747


In [105]:
# Testing
total, correct = 0, 0
with torch.no_grad():
    for data in testloader:
        inputs, labels = data
        inputs = inputs.to('cuda')
        labels = labels.to('cuda')

        outputs = lstm(inputs)

        _, predicted = torch.max(outputs, 1)

        total += labels.size(0)
        correct += (predicted == labels).sum()

print(f'Accuracy of the model is: {100*correct/total:.2f}%')

Accuracy of the model is: 26.93%
